In [41]:
import numpy as np

import matplotlib.pyplot as plt

from itertools import product

In [213]:
def generate_board(alpha = 0.9):
    a = np.full([8,8], ' ')

    pieces = ["♔", "♚"] + list(np.random.permutation(["♕","♗","♗","♘","♘","♖","♖"] + 
                                                     ["♛","♝","♝","♞","♞","♜","♜"]))
    k = 0

    pawns = np.random.permutation(["♙","♙","♙","♙","♙","♙","♙","♙"] + ["♟","♟","♟","♟","♟","♟","♟","♟"])
    m = 0

    cells = np.random.permutation(list(product(range(8), range(8))))
    
    for i, j in cells:
        if k < 2 or k < len(pieces) and np.random.rand() > 0.5 and np.random.rand() > alpha:
            a[i, j] = pieces[k]
            k += 1
        elif m < len(pawns) and i > 0 and i < 7 and np.random.rand() > alpha:
            a[i, j] = pawns[m]
            m += 1

    return a        

In [4]:
fig2img = {
    "♔": "kw",
    "♕": "qw",
    "♗": "bw",
    "♘": "nw",
    "♖": "rw",
    "♙": "pw",
    "♚": "kb",
    "♛": "qb",
    "♝": "bb",
    "♞": "nb",
    "♜": "rb",
    "♟": "pb",
    " ": ""
}

In [5]:
def get_square_color(i, j):
    return "b" if (i + j) % 2 else "w"

In [6]:
def get_image_name(a, i, j):
    return "images/" + fig2img[a[i, j]] + get_square_color(i, j) + ".png"

In [7]:
def get_board_image(a):
    b = np.empty((0, 30 * 8, 3))
    for i in range(len(a)):
        r = np.empty((30, 0, 3))
        for j in range(len(a[0])):
            r = np.hstack([r, plt.imread(get_image_name(a, i, j))])
        b = np.vstack([b, r])
    return b

In [258]:
b = generate_board()
a = get_board_image(b)
plt.imsave("board.png", a)

from IPython.display import Image
from datetime import datetime

print(b)

Image(url="board.png?timestamp=" + str(datetime.now().timestamp()))

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' '♔' '♙' '♟' ' ' '♟' ' ']
 [' ' '♙' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' '♙' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' '♟' '♟' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 ['♗' ' ' ' ' ' ' ' ' '♝' ' ' '♙']
 ['♚' ' ' ' ' ' ' ' ' ' ' ' ' ' ']]


In [238]:
from keras.layers import Input, Dense
from keras.models import Model

ModuleNotFoundError: No module named 'keras'